Modules

In [188]:
import torch
import torch.nn.functional as F
import numpy as np

Dataset

In [14]:
file = open(r'dataset\wizard_of_oz.txt','r',encoding='utf-8')
text = file.read()
file.close()
len(text)

252051

In [30]:
tokens = sorted(set(text))

In [29]:
vocabulary = {token:index for index,token in enumerate(tokens)}
vocabulary_index = {index:token for index,token in enumerate(tokens)}

In [31]:
def vocabulary_encoder(sequence):
    return [vocabulary[s] for s in sequence]

def vocabulary_decoder(sequence):
    return ''.join([vocabulary_index[s] for s in sequence])


Pre-Processing

In [38]:
data = torch.tensor(vocabulary_encoder(text),dtype=torch.long)

In [163]:
n = int(.8*len(data))
train_data = data[:n]
test_data = data[n:]
block_size, batch_size = 8, 4

In [240]:
def get_batch(dataset):
    ix = torch.randint(len(dataset)-block_size, (batch_size,))
    x = torch.stack([dataset[i:i+block_size] for i in ix])
    y = torch.stack([dataset[i+1:i+block_size+1] for i in ix])
    # x, y = x.to(device)
    return x, y

In [241]:
x, y = get_batch(train_data)

In [150]:
print('input: ', x)
print('labels: ', y)

input:  tensor([[ 1, 64, 75, 78, 63, 63,  1, 77, 72, 71],
        [58, 71, 61,  1, 80, 58, 77, 62, 75, 62],
        [62,  1, 65, 62, 75, 62, 28,  3,  1, 58],
        [ 3, 48, 65, 58, 77,  1, 66, 76,  1, 75],
        [77, 65, 62,  1, 76, 78, 60, 60, 62, 76],
        [62, 75,  1, 75, 72, 80,  1, 72, 63,  1],
        [ 1, 64, 66, 75, 69, 14,  0,  0,  3, 44],
        [ 1, 59, 58, 61, 69, 82,  1, 63, 75, 66],
        [76,  1, 77, 72,  1, 60, 75, 58, 60, 68],
        [ 1, 58,  1, 69, 58, 71, 61, 66, 71, 64]])
labels:  tensor([[64, 75, 78, 63, 63,  1, 77, 72, 71],
        [71, 61,  1, 80, 58, 77, 62, 75, 62],
        [ 1, 65, 62, 75, 62, 28,  3,  1, 58],
        [48, 65, 58, 77,  1, 66, 76,  1, 75],
        [65, 62,  1, 76, 78, 60, 60, 62, 76],
        [75,  1, 75, 72, 80,  1, 72, 63,  1],
        [64, 66, 75, 69, 14,  0,  0,  3, 44],
        [59, 58, 61, 69, 82,  1, 63, 75, 66],
        [ 1, 77, 72,  1, 60, 75, 58, 60, 68],
        [58,  1, 69, 58, 71, 61, 66, 71, 64]])


Model

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [242]:
vocab_size = len(vocabulary)

In [247]:
class Model(torch.nn.Module):
    def __init__(self,vocab_size,embedding_dim):
        super(Model,self).__init__()
        self.token_embedding_layer = torch.nn.Embedding(vocab_size,embedding_dim)

    def forward(self,input,targets=None):
        logits = self.token_embedding_layer(input)
        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits,loss
    
    def generate(self, input, max_new_tokens):
        for i in range(max_new_tokens):
            logits, loss = self.forward(input)
            logits = logits[:,-1,:]
            probabilities = F.softmax(logits,dim=-1)
            input_next = torch.multinomial(probabilities,num_samples=1)
            input = torch.cat((input,input_next),dim=1)
        return input


In [248]:
model = Model(vocab_size, vocab_size)
model = model.to(device)
print(model)

Model(
  (token_embedding_layer): Embedding(92, 92)
)


In [245]:
optimizer = torch.optim.AdamW(model.parameters(),lr=.001)

In [249]:
for epoch in range(100):
    xb, yb = get_batch(train_data)
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss)

tensor(5.1547, grad_fn=<NllLossBackward0>)
